<a href="https://colab.research.google.com/github/mahekgheewala/Model_Selection/blob/main/Model_selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importing Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

In [3]:
heart_data = pd.read_csv('/content/heart (1).csv')

In [4]:
heart_data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [5]:
heart_data.shape

(303, 14)

In [6]:
heart_data.isnull().sum()

,0
age,0
sex,0
cp,0
trestbps,0
chol,0
fbs,0
restecg,0
thalach,0
exang,0
oldpeak,0


In [8]:
heart_data['target'].value_counts()

,count
target,
1,165
0,138


1---> defective

0---> not defective

In [9]:
# Splitting X and Y
X = heart_data.drop(columns='target', axis = 1)
Y = heart_data['target']

In [10]:
print(X)
print(Y)

     age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  \
0     63    1   3       145   233    1        0      150      0      2.3   
1     37    1   2       130   250    0        1      187      0      3.5   
2     41    0   1       130   204    0        0      172      0      1.4   
3     56    1   1       120   236    0        1      178      0      0.8   
4     57    0   0       120   354    0        1      163      1      0.6   
..   ...  ...  ..       ...   ...  ...      ...      ...    ...      ...   
298   57    0   0       140   241    0        1      123      1      0.2   
299   45    1   3       110   264    0        1      132      0      1.2   
300   68    1   0       144   193    1        1      141      0      3.4   
301   57    1   0       130   131    0        1      115      1      1.2   
302   57    0   1       130   236    0        0      174      0      0.0   

     slope  ca  thal  
0        0   0     1  
1        0   0     2  
2        2   0    

In [11]:
X = np.asarray(X)
Y = np.asarray(Y)

# **Model Selection**

1. Comparing the models with default hyperparameters values using cross validation.

In [12]:
models = [LogisticRegression(max_iter=1000), SVC(kernel='linear'), KNeighborsClassifier(), RandomForestClassifier(random_state=0)]
# we are using max_iter because we have not standardized the data, hence it would give us an error

In [13]:
def compare_models_cross_validation():

  for model in models:
    cv_score = cross_val_score(model, X,Y,cv=5)
    mean_accuracy = sum(cv_score)/len(cv_score)
    mean_accuracy = mean_accuracy * 100
    mean_accuracy = round(mean_accuracy, 2)
    print(mean_accuracy, model)
    # print(cv_score) ----> it will show all 5 accuracy

In [14]:
compare_models_cross_validation()

82.83 LogisticRegression(max_iter=1000)
82.83 SVC(kernel='linear')
64.39 KNeighborsClassifier()
83.81 RandomForestClassifier(random_state=0)


# Model Selection with hyperparameter Tuning

2. We would give different hyperparameter values and then check the accuracy.

In [43]:
models_list = [LogisticRegression(max_iter=5000), SVC(), KNeighborsClassifier(), RandomForestClassifier(random_state=0)]

In [44]:
#  Creating a dictionary with all hyperparameters values
hyperparameters = {
    'logistic': {
        'C': [1,5,10,20]
    },
    'SVC' : {
        'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
        'C': [1,5,10,20]
    },
    'KNeighbour': {
        # n neighbours mean the neighbours we consider for creating a group
        'n_neighbors': [3,5,10]
    },
    'Random_Forest' : {
        # no. of decision trees we want to make
        'n_estimators' : [10,20,50,100]
    }
}

In [45]:
type(hyperparameters)

dict

In [46]:
print(hyperparameters.keys())

dict_keys(['logistic', 'SVC', 'KNeighbour', 'Random_Forest'])


In [48]:
print(hyperparameters.values())

dict_values([{'C': [1, 5, 10, 20]}, {'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'C': [1, 5, 10, 20]}, {'n_neighbors': [3, 5, 10]}, {'n_estimators': [10, 20, 50, 100]}])


Converting key into lists

In [49]:
#  CONVERTING KEYS INTO LIST
model_keys = list(hyperparameters.keys())
print(model_keys)

['logistic', 'SVC', 'KNeighbour', 'Random_Forest']


In [50]:
# ACCESSING THE KEYS
# just write the data from list you want into square brackets
hyperparameters['logistic']

{'C': [1, 5, 10, 20]}

In [51]:
model_keys[0]
# if i write this, then first element would be printed

'logistic'

the above prints the key name, the below prints the key value

In [52]:
hyperparameters[model_keys[0]]

{'C': [1, 5, 10, 20]}

# Applying GridSearchCV

In [56]:
def ModelSelection(models, hyperparameters_dict):

# in result we would be storing all the best values
  result = []
# we have assigned i=0, so we can access model_keys.....the list(in line 36,37)
  i=0
  for model in models:
    key = model_keys[i]

    # here we are going to pass the parameters of models, one by one
    params = hyperparameters_dict[key]
    i+=1
    print(model)
    print(params)

    classifier = GridSearchCV(model, params, cv=5)
    # Fitting the data
    classifier.fit(X,Y)

    # now saving new values to result
    result.append(
        {'model_used': model,
         'highest_score': classifier.best_score_,
         'best_hyperparameters': classifier.best_params_
         }
    )
  result_data = pd.DataFrame(result, columns=['model_used', 'highest_score', 'best_hyperparameters'])
  return result_data

In [57]:
ModelSelection(models_list, hyperparameters)

LogisticRegression(max_iter=5000)
{'C': [1, 5, 10, 20]}
SVC()
{'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'C': [1, 5, 10, 20]}
KNeighborsClassifier()
{'n_neighbors': [3, 5, 10]}
RandomForestClassifier(random_state=0)
{'n_estimators': [10, 20, 50, 100]}


,model_used,highest_score,best_hyperparameters
0,LogisticRegression(max_iter=5000),0.831585,{'C': 5}
1,SVC(),0.828306,"{'C': 1, 'kernel': 'linear'}"
2,KNeighborsClassifier(),0.643880,{'n_neighbors': 5}
3,RandomForestClassifier(random_state=0),0.838087,{'n_estimators': 100}
